In [1]:
import os 
os.environ["SPARK_HOME"] = ""

import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import col, avg

spark = SparkSession.builder\
    .appName("capstone_project")\
    .enableHiveSupport()\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/05/24 00:22:18 WARN Utils: Your hostname, pcminh-ThinkPad resolves to a loopback address: 127.0.1.1; using 172.31.163.12 instead (on interface eth0)
23/05/24 00:22:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/24 00:22:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [3]:
from schemas import redlight_cam_schema, speed_cam_schema, streets_schema, traffic_hist_schema

tables = [
    'crashes', 
    'redlight_cam', 
    'speed_cam', 
    'streets', 
    'traffic_hist'
]

paths = {
    table: f'./data/raw/traffic/{table}/{table}.csv'
    for table in tables
} 

schemas = {
    # 'crashes'
    'redlight_cam': redlight_cam_schema,
    'speed_cam': speed_cam_schema,
    'streets': streets_schema,
    'traffic_hist': traffic_hist_schema
}

def read_table_to_view(spark: SparkSession):
    for table in tables:
        read_obj = spark.read.format("csv").option("header", "true")

        if table in schemas:
            read_obj = read_obj.schema(schemas[table])
        else:
            read_obj = read_obj.option("inferSchema", "true")
 
        df = read_obj.load(paths[table])
        df.createOrReplaceTempView(table)

In [4]:
read_table_to_view(spark)

23/05/24 00:22:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [66]:
spark.sql("""
    with f_congestion_batch_staging
    as (
        select 
            CAST(format_string('%04d', segment_id) AS string) as segment_padded,
            to_timestamp(time, 'MM/dd/yyyy HH:mm:ss a') as timestamp,
            date_format(timestamp + INTERVAL 5 HOURS, 'yyyyMMddhhmm') as gmtts_str,
            CONCAT_WS('-', segment_padded, gmtts_str) AS congestion_key,
            segment_id,
            speed,
            bus_count,
            message_count,
            CAST(date_format(timestamp, 'yyyyMMdd') AS INT) as date_key,
            CAST(date_format(timestamp, 'hh') AS INT) as hour_key
        from traffic_hist
    )

    select 
        congestion_key as congestion_record_id,
        segment_id, 
        timestamp,
        gmtts_str,
        speed,
        bus_count,
        message_count,
        date_key,
        hour_key
    from f_congestion_batch_staging
    where speed > -1
""").show()

+--------------------+----------+-------------------+------------+-----+---------+-------------+--------+--------+
|congestion_record_id|segment_id|          timestamp|   gmtts_str|speed|bus_count|message_count|date_key|hour_key|
+--------------------+----------+-------------------+------------+-----+---------+-------------+--------+--------+
|   0260-202304051120|       260|2023-04-05 06:20:57|202304051120|   23|        2|           27|20230405|       6|
|   0687-202304051121|       687|2023-04-05 06:21:04|202304051121|   21|        2|           26|20230405|       6|
|   0470-202304051121|       470|2023-04-05 06:21:09|202304051121|   27|        3|           24|20230405|       6|
|   0102-202304051120|       102|2023-04-05 06:20:43|202304051120|   29|        1|            2|20230405|       6|
|   0373-202304051120|       373|2023-04-05 06:20:47|202304051120|   22|        1|            5|20230405|       6|
|   1186-202304051121|      1186|2023-04-05 06:21:38|202304051121|   31|        

In [68]:
spark.sql("""
    select 
        crash_record_id as crash_record_id,
        rd_no as report_number,
        to_timestamp(crash_date, 'MM/dd/yyyy HH:mm:ss a') as crash_timestamp,
        CAST(date_format(crash_timestamp, 'yyyyMMdd') AS INT) as crash_date_key,
        CAST(date_format(crash_timestamp, 'hh') AS INT) as crash_hour_key,
        posted_speed_limit as posted_speed_limit,
        traffic_control_device as traffic_control_device,
        device_condition as device_condition,
        weather_condition as weather_condition,
        lighting_condition as lighting_condition,
        roadway_surface_cond as roadway_surface_cond,
        road_defect as road_defect,
        first_crash_type as first_crash_type,
        trafficway_type as trafficway_type,
        lane_cnt as lane_cnt,
        report_type as report_type,
        crash_type as crash_type,
        damage as damage,
        street_no as street_no,
        ds.street_key as street_key,
        num_units as num_units,
        most_severe_injury as most_severy_injury,
        injuries_total as injuries_total,
        injuries_fatal as injuries_fatal,
        injuries_incapacitating as injuries_incapacitating,
        injuries_non_incapacitating as injuries_non_incapacitating,
        injuries_reported_not_evident as injuries_reported_not_evident,
        injuries_no_indication as injuries_no_indication,
        injuries_unknown as injuries_unknown,
        latitude as latitude,
        longitude as longitude
    FROM crashes c
    LEFT JOIN d_street ds 
    ON ds.full_street_name LIKE CONCAT(UPPER(c.street_name), '%');
""").show()

+--------------------+-------------+-------------------+--------------+--------------+------------------+----------------------+--------------------+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------+--------------------+--------------------+-------------+---------+----------+---------+--------------------+--------------+--------------+-----------------------+---------------------------+-----------------------------+----------------------+----------------+------------+-------------+
|     crash_record_id|report_number|    crash_timestamp|crash_date_key|crash_hour_key|posted_speed_limit|traffic_control_device|    device_condition|weather_condition|  lighting_condition|roadway_surface_cond|road_defect|    first_crash_type|     trafficway_type|lane_cnt|         report_type|          crash_type|       damage|street_no|street_key|num_units|  most_severy_injury|injuries_total|injuries_fatal|injuries_incapacitating|injuri